In [1]:
import torch
import numpy as np
import pandas as pd
import torch.nn.functional as F
import os

In [15]:
def load_csv_data(input_folder: str,
                  train_fname: str,
                  test_fname: str):
    """
    Reads train & test CSVs from disk.
    
    Returns:
      train_df, test_df (both pandas.DataFrame)
    """
    train_path = os.path.join(input_folder, train_fname)
    test_path  = os.path.join(input_folder, test_fname)
    train_df = pd.read_csv(train_path)
    test_df  = pd.read_csv(test_path)
    return train_df, test_df

def extract_features_labels(df: pd.DataFrame):
    """
    Splits a DataFrame into numpy feature array X and label vector y.
    
    The last column is the label.
    """
    X = df.iloc[:, :-1].values
    y = df.iloc[:,  -1].values
    return X, y

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load data
input_folder = '/home/zyang44/Github/baseline_cicIOT/P1_structurelevel/efficiency/input_files'
train_fname = 'logiKNet_train_35945.csv'
test_fname = 'logiKNet_test_3994.csv'

train_df, test_df = load_csv_data(input_folder, train_fname, test_fname)
# Extract features and labels   
X_train, y_train = extract_features_labels(train_df)
X_test, y_test = extract_features_labels(test_df)

# dataset = {
#     'train_input': torch.tensor(X_train, dtype=torch.float32, device=device),
#     'train_label': F.one_hot(torch.tensor(y_train, dtype=torch.long, device=device), num_classes=6),
#     'test_input': torch.tensor(X_test, dtype=torch.float32, device=device),
#     'test_label': F.one_hot(torch.tensor(y_test, dtype=torch.long, device=device), num_classes=6)
# }

dataset_numeric = {
    'train_input': torch.tensor(X_train, dtype=torch.float32, device=device),
    'train_label': torch.tensor(y_train, dtype=torch.long, device=device),
    'test_input': torch.tensor(X_test, dtype=torch.float32, device=device),
    'test_label': torch.tensor(y_test, dtype=torch.long, device=device)
}


In [ ]:
# this is a standard PyTorch DataLoader to load the dataset for the training and testing of the model
class DataLoader(object):
    def __init__(self,
                 data,
                 labels,
                 batch_size=1,
                 shuffle=True):
        self.data = data
        self.labels = labels
        self.batch_size = batch_size
        self.shuffle = shuffle

    def __len__(self):
        return int(np.ceil(self.data.shape[0] / self.batch_size))

    def __iter__(self):
        n = self.data.shape[0]
        idxlist = list(range(n))
        if self.shuffle:
            np.random.shuffle(idxlist)

        for _, start_idx in enumerate(range(0, n, self.batch_size)):
            end_idx = min(start_idx + self.batch_size, n)
            data = self.data[idxlist[start_idx:end_idx]]
            labels = self.labels[idxlist[start_idx:end_idx]]
            ############################################################
            # Check if any class is missing in the batch
            # present_classes = np.unique(labels.cpu().numpy())
            # all_classes = np.arange(len(label_mapping))  # Adjust based on number of classes
            # missing_classes = set(all_classes) - set(present_classes)
            #
            # if missing_classes:
            #     print(f"Batch {start_idx // self.batch_size} is missing classes {missing_classes}")
            ############################################################
            yield data, labels


train_loader = DataLoader(
    dataset_numeric['train_input'],
    dataset_numeric['train_label'], 
    batch_size=len(X_train), 
    shuffle=True
    )
test_loader = DataLoader(
    dataset_numeric['test_input'],
    dataset_numeric['test_label'],
    # batch_size=len(X_test), 
    batch_size=1,
    shuffle=False
    )

In [17]:
class LogitsToPredicate(torch.nn.Module):
    """
    This model has inside a logits model, that is a model which compute logits for the classes given an input example x.
    The idea of this model is to keep logits and probabilities separated. The logits model returns the logits for an example,
    while this model returns the probabilities given the logits model.

    In particular, it takes as input an example x and a class label l. It applies the logits model to x to get the logits.
    Then, it applies a softmax function to get the probabilities per classes. Finally, it returns only the probability related
    to the given class l.
    """

    def __init__(self, logits_model):
        super(LogitsToPredicate, self).__init__()
        self.logits_model = logits_model
        self.softmax = torch.nn.Softmax(dim=1)

    def forward(self, x, l, training=False):
        logits = self.logits_model(x, training=training)
        probs = self.softmax(logits)
        out = torch.sum(probs * l, dim=1)  # 计算并返回与给定类标签l对应的概率值
        return out


class MLP(torch.nn.Module):
    """
    This model returns the logits for the classes given an input example. It does not compute the softmax, so the output
    are not normalized.
    This is done to separate the accuracy computation from the satisfaction level computation. Go through the example
    to understand it.
    """

    def __init__(self, layer_sizes):
        super(MLP, self).__init__()
        self.elu = torch.nn.ELU()
        self.dropout = torch.nn.Dropout(0.2)
        self.linear_layers = torch.nn.ModuleList([torch.nn.Linear(layer_sizes[i - 1], layer_sizes[i])
                                                  for i in range(1, len(layer_sizes))])

    def forward(self, x, training=False):
        """
        Method which defines the forward phase of the neural network for our multi class classification task.
        In particular, it returns the logits for the classes given an input example.

        :param x: the features of the example
        :param training: whether the network is in training mode (dropout applied) or validation mode (dropout not applied)
        :return: logits for example x
        """
        for layer in self.linear_layers[:-1]:
            x = self.elu(layer(x))
            if training:
                x = self.dropout(x)
        logits = self.linear_layers[-1](x)
        return logits


class MultiKANModel(torch.nn.Module):
    def __init__(self, kan):
        """
        Wrap an already built MultKAN instance.
        Args:
            kan: a MultKAN model (which has attributes such as act_fun, symbolic_fun, node_bias, node_scale,
                 subnode_bias, subnode_scale, depth, width, mult_homo, mult_arity, input_id, symbolic_enabled, etc.)
        """
        super(MultiKANModel, self).__init__()
        self.kan = kan

    def forward(self, x, training=False, singularity_avoiding=False, y_th=10.):
        # Select input features according to input_id
        x = x[:, self.kan.input_id.long()]
        # Loop through each layer
        for l in range(self.kan.depth):
            # Get outputs from the numerical branch (KANLayer) of current layer
            x_numerical, preacts, postacts_numerical, postspline = self.kan.act_fun[l](x)
            # Get output from the symbolic branch if enabled
            if self.kan.symbolic_enabled:
                x_symbolic, postacts_symbolic = self.kan.symbolic_fun[l](x, singularity_avoiding=singularity_avoiding, y_th=y_th)
            else:
                x_symbolic = 0.
            # Sum the numerical and symbolic outputs
            x = x_numerical + x_symbolic

            # Subnode affine transformation
            x = self.kan.subnode_scale[l][None, :] * x + self.kan.subnode_bias[l][None, :]

            # Process multiplication nodes
            dim_sum = self.kan.width[l+1][0]
            dim_mult = self.kan.width[l+1][1]
            if dim_mult > 0:
                if self.kan.mult_homo:
                    for i in range(self.kan.mult_arity-1):
                        if i == 0:
                            x_mult = x[:, dim_sum::self.kan.mult_arity] * x[:, dim_sum+1::self.kan.mult_arity]
                        else:
                            x_mult = x_mult * x[:, dim_sum+i+1::self.kan.mult_arity]
                else:
                    for j in range(dim_mult):
                        acml_id = dim_sum + int(np.sum(self.kan.mult_arity[l+1][:j]))
                        for i in range(self.kan.mult_arity[l+1][j]-1):
                            if i == 0:
                                x_mult_j = x[:, [acml_id]] * x[:, [acml_id+1]]
                            else:
                                x_mult_j = x_mult_j * x[:, [acml_id+i+1]]
                        if j == 0:
                            x_mult = x_mult_j
                        else:
                            x_mult = torch.cat([x_mult, x_mult_j], dim=1)
                # Concatenate sum and mult parts
                x = torch.cat([x[:, :dim_sum], x_mult], dim=1)

            # Node affine transformation
            x = self.kan.node_scale[l][None, :] * x + self.kan.node_bias[l][None, :]

        # Final x corresponds to the logits output of the whole model
        return x

In [18]:

def compute_accuracy(loader, model):
    total_correct = 0
    total_samples = 0
    for data, labels in loader:
        logits = model(data)
        preds = torch.argmax(logits, dim=1)
        total_correct += (preds == labels).sum()
        total_samples += labels.numel()
    return total_correct.float() / total_samples


def save_model(model, model_save_folder, model_name):
    """
    Save the model to disk.
    """
    torch.save(model.state_dict(), os.path.join(model_save_folder, model_name))

    print(f"Model saved to {os.path.join(model_save_folder, model_name)}")


model_state_folder = '/home/zyang44/Github/baseline_cicIOT/P1_structurelevel/efficiency/model_weights'


**MLP with CrossEntropyLoss**

In [23]:
#  Define the MLP predicate
mlp = MLP(layer_sizes=(18, 10, 6)).to(device)

# MLP with standard loss fn 
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.001)

# Train the model
for epoch in range(401):
    for data, labels in train_loader:               # iterate batches
        optimizer.zero_grad()
        logits = mlp(data, training=True)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
    # test
    acc = compute_accuracy(test_loader, mlp)
    print(f"Epoch {epoch}, Loss: {loss.item()}, Test accuracy: {acc.item()}")

# Save the model
save_model(mlp, model_state_folder, 'mlp.pt')

Epoch 0, Loss: 1.8321869373321533, Test accuracy: 0.22033049166202545
Epoch 1, Loss: 1.8269691467285156, Test accuracy: 0.22258387506008148
Epoch 2, Loss: 1.8208402395248413, Test accuracy: 0.2248372584581375
Epoch 3, Loss: 1.8149583339691162, Test accuracy: 0.2265898883342743
Epoch 4, Loss: 1.8083497285842896, Test accuracy: 0.22934401035308838
Epoch 5, Loss: 1.8035719394683838, Test accuracy: 0.2315973937511444
Epoch 6, Loss: 1.7972102165222168, Test accuracy: 0.2343515306711197
Epoch 7, Loss: 1.7936214208602905, Test accuracy: 0.23710565268993378
Epoch 8, Loss: 1.787880539894104, Test accuracy: 0.23960942029953003
Epoch 9, Loss: 1.7805042266845703, Test accuracy: 0.24236354231834412
Epoch 10, Loss: 1.7773271799087524, Test accuracy: 0.24436655640602112
Epoch 11, Loss: 1.7715272903442383, Test accuracy: 0.24636955559253693
Epoch 12, Loss: 1.767098307609558, Test accuracy: 0.24862293899059296
Epoch 13, Loss: 1.761338472366333, Test accuracy: 0.25137707591056824
Epoch 14, Loss: 1.75610

In [ ]:
def load_model_state(infer_model, model_save_folder, model_name):
    """
    Load the model from disk.
    """
    checkpoint = torch.load(os.path.join(model_save_folder, model_name), 
                            map_location=device)
    infer_model.load_state_dict(checkpoint)
    infer_model.eval()
    return infer_model

# Load the model
mlp_infer = MLP(layer_sizes=(18, 10, 6)).to(device)
mlp_infer = load_model_state(mlp_infer, model_state_folder, 'mlp.pt')
# Test the model
with torch.no_grad():
    for data, labels in test_loader:
        logits = mlp_infer(data)
        preds = torch.argmax(logits, dim=1)
        print(f"Predictions: {preds}, Labels: {labels}")
        
        mlp_infer_acc = compute_accuracy(test_loader, mlp_infer)
        print(f"Test accuracy of the loaded model: {mlp_infer_acc.item()}")
        break

**LTN utils setup**

In [19]:
import ltn
import ltn.fuzzy_ops
import torch
import os
from kan import KAN

# define the connectives, quantifiers, and the SatAgg
Not = ltn.Connective(ltn.fuzzy_ops.NotStandard())
And = ltn.Connective(ltn.fuzzy_ops.AndProd())   # And = ltn.Connective(custom_fuzzy_ops.AndProd())
Or = ltn.Connective(ltn.fuzzy_ops.OrProbSum())
Forall = ltn.Quantifier(ltn.fuzzy_ops.AggregPMeanError(p=2), quantifier="f")
Exists = ltn.Quantifier(ltn.fuzzy_ops.AggregPMean(p=2), quantifier="e")
Implies = ltn.Connective(ltn.fuzzy_ops.ImpliesReichenbach())
SatAgg = ltn.fuzzy_ops.SatAgg()

# define ltn constants
l_MQTT_DDoS_Connect_Flood = ltn.Constant(torch.tensor([1, 0, 0, 0, 0, 0]))
l_MQTT_DDoS_Publish_Flood = ltn.Constant(torch.tensor([0, 1, 0, 0, 0, 0]))
l_MQTT_DoS_Connect_Flood = ltn.Constant(torch.tensor([0, 0, 1, 0, 0, 0]))
l_MQTT_DoS_Publish_Flood = ltn.Constant(torch.tensor([0, 0, 0, 1, 0, 0]))
l_MQTT_Malformed_Data = ltn.Constant(torch.tensor([0, 0, 0, 0, 1, 0]))
l_Benign = ltn.Constant(torch.tensor([0, 0, 0, 0, 0, 1]))

**LogiK-mlp** 
: MLP with LTN

In [25]:
def compute_sat_levels(loader, P):
	sat_level  = 0
	for data, labels in loader:
		x = ltn.Variable("x", data)
		x_MQTT_DDoS_Connect_Flood = ltn.Variable("x_MQTT_DDoS_Connect_Flood", data[labels == 0])
		x_MQTT_DDoS_Publish_Flood = ltn.Variable("x_MQTT_DDoS_Publish_Flood", data[labels == 1])
		x_MQTT_DoS_Connect_Flood = ltn.Variable("x_MQTT_DoS_Connect_Flood", data[labels == 2])
		x_MQTT_DoS_Publish_Flood = ltn.Variable("x_MQTT_DoS_Publish_Flood", data[labels == 3])
		x_MQTT_Malformed_Data = ltn.Variable("x_MQTT_Malformed_Data", data[labels == 4])
		x_Benign = ltn.Variable("x_Benign", data[labels == 5])

		sat_level = SatAgg(
			Forall(x_MQTT_DDoS_Connect_Flood, P(x_MQTT_DDoS_Connect_Flood, l_MQTT_DDoS_Connect_Flood)),
			Forall(x_MQTT_DDoS_Publish_Flood, P(x_MQTT_DDoS_Publish_Flood, l_MQTT_DDoS_Publish_Flood)),
			Forall(x_MQTT_DoS_Connect_Flood, P(x_MQTT_DoS_Connect_Flood, l_MQTT_DoS_Connect_Flood)),
			Forall(x_MQTT_DoS_Publish_Flood, P(x_MQTT_DoS_Publish_Flood, l_MQTT_DoS_Publish_Flood)),
			Forall(x_MQTT_Malformed_Data, P(x_MQTT_Malformed_Data, l_MQTT_Malformed_Data)),
			Forall(x_Benign, P(x_Benign, l_Benign))
		)
	return sat_level

mlp = MLP(layer_sizes=(18, 10, 6)).to(device)
P_mlp = ltn.Predicate(LogitsToPredicate(mlp))

optimizer = torch.optim.Adam(P_mlp.parameters(), lr=0.001)

# Train the model
for epoch in range(801):             
	optimizer.zero_grad()
	sat_mlp = compute_sat_levels(train_loader, P_mlp)
	loss = 1. - sat_mlp
	loss.backward()
	optimizer.step()
	train_loss_mlp  = loss.item()
	
	# test
	test_acc_mlp = compute_accuracy(test_loader, mlp)
	test_sat_mlp = compute_sat_levels(test_loader, P_mlp)
	print(f"Epoch {epoch} | Logic-MLP (loss/acc/sat): {train_loss_mlp:.3f}/{test_acc_mlp:.3f}/{sat_mlp:.3f}({test_sat_mlp:.3f})")

print("Training finished.")

# save the models as .pt files
save_model(mlp, model_state_folder, 'logic_mlp.pt')

Epoch 0 | Logic-MLP (loss/acc/sat): 0.840/0.156/0.160(0.161)
Epoch 1 | Logic-MLP (loss/acc/sat): 0.840/0.153/0.160(0.161)
Epoch 2 | Logic-MLP (loss/acc/sat): 0.839/0.147/0.161(0.162)
Epoch 3 | Logic-MLP (loss/acc/sat): 0.838/0.136/0.162(0.163)
Epoch 4 | Logic-MLP (loss/acc/sat): 0.837/0.141/0.163(0.164)
Epoch 5 | Logic-MLP (loss/acc/sat): 0.836/0.263/0.164(0.165)
Epoch 6 | Logic-MLP (loss/acc/sat): 0.836/0.264/0.164(0.166)
Epoch 7 | Logic-MLP (loss/acc/sat): 0.835/0.263/0.165(0.167)
Epoch 8 | Logic-MLP (loss/acc/sat): 0.834/0.264/0.166(0.167)
Epoch 9 | Logic-MLP (loss/acc/sat): 0.833/0.264/0.167(0.168)
Epoch 10 | Logic-MLP (loss/acc/sat): 0.832/0.263/0.168(0.169)
Epoch 11 | Logic-MLP (loss/acc/sat): 0.831/0.263/0.169(0.170)
Epoch 12 | Logic-MLP (loss/acc/sat): 0.830/0.263/0.170(0.171)
Epoch 13 | Logic-MLP (loss/acc/sat): 0.830/0.263/0.170(0.172)
Epoch 14 | Logic-MLP (loss/acc/sat): 0.829/0.263/0.171(0.173)
Epoch 15 | Logic-MLP (loss/acc/sat): 0.828/0.263/0.172(0.174)
Epoch 16 | Logic-M

In [27]:
def load_model_state(infer_model, model_save_folder, model_name):
    """
    Load the model from disk.
    """
    checkpoint = torch.load(os.path.join(model_save_folder, model_name), 
                            map_location=device)
    infer_model.load_state_dict(checkpoint)
    infer_model.eval()
    return infer_model

# Load the model
logicmlp_infer = MLP(layer_sizes=(18, 10, 6)).to(device)
logicmlp_infer = load_model_state(logicmlp_infer, model_state_folder, 'logic_mlp.pt')
# Test the model
with torch.no_grad():
    for data, labels in test_loader:
        logits = logicmlp_infer(data)
        preds = torch.argmax(logits, dim=1)
        print(f"Predictions: {preds}, Labels: {labels}")
        
        logicmlp_infer_acc = compute_accuracy(test_loader, logicmlp_infer)
        print(f"Test accuracy of the loaded model: {logicmlp_infer_acc.item()}")
        break

Predictions: tensor([2, 2, 1,  ..., 2, 4, 2]), Labels: tensor([2, 0, 0,  ..., 2, 4, 2])
Test accuracy of the loaded model: 0.6647471189498901


/tmp/ipykernel_1302988/1958631697.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(os.path.join(model_save_folder, model_name),


**LogiKNet**

In [ ]:

def compute_sat_levels(loader, P):
	sat_level  = 0
	for data, labels in loader:
		x = ltn.Variable("x", data)
		x_MQTT_DDoS_Connect_Flood = ltn.Variable("x_MQTT_DDoS_Connect_Flood", data[labels == 0])
		x_MQTT_DDoS_Publish_Flood = ltn.Variable("x_MQTT_DDoS_Publish_Flood", data[labels == 1])
		x_MQTT_DoS_Connect_Flood = ltn.Variable("x_MQTT_DoS_Connect_Flood", data[labels == 2])
		x_MQTT_DoS_Publish_Flood = ltn.Variable("x_MQTT_DoS_Publish_Flood", data[labels == 3])
		x_MQTT_Malformed_Data = ltn.Variable("x_MQTT_Malformed_Data", data[labels == 4])
		x_Benign = ltn.Variable("x_Benign", data[labels == 5])

		sat_level = SatAgg(
			Forall(x_MQTT_DDoS_Connect_Flood, P(x_MQTT_DDoS_Connect_Flood, l_MQTT_DDoS_Connect_Flood)),
			Forall(x_MQTT_DDoS_Publish_Flood, P(x_MQTT_DDoS_Publish_Flood, l_MQTT_DDoS_Publish_Flood)),
			Forall(x_MQTT_DoS_Connect_Flood, P(x_MQTT_DoS_Connect_Flood, l_MQTT_DoS_Connect_Flood)),
			Forall(x_MQTT_DoS_Publish_Flood, P(x_MQTT_DoS_Publish_Flood, l_MQTT_DoS_Publish_Flood)),
			Forall(x_MQTT_Malformed_Data, P(x_MQTT_Malformed_Data, l_MQTT_Malformed_Data)),
			Forall(x_Benign, P(x_Benign, l_Benign))
		)
	return sat_level
    
# Define myKAN predicate
kan = KAN(width=[18, 10, 6], grid=5, k=3, seed=42, device=device)
P_kan = ltn.Predicate(LogitsToPredicate(MultiKANModel(kan)))

optimizer_kan = torch.optim.Adam(P_kan.parameters(), lr=0.001)

for epoch in range(401):
	optimizer_kan.zero_grad()
	sat_kan = compute_sat_levels(train_loader, P_kan)
	loss = 1. - sat_kan
	loss.backward()
	optimizer_kan.step()
	train_loss_kan = loss.item()

	# Test the KAN
	test_acc_kan = compute_accuracy(test_loader, kan)
	test_sat_kan = compute_sat_levels(test_loader, P_kan)

	print(f"Epoch {epoch} | KAN (loss/acc/sat): {train_loss_kan:.3f}/{test_acc_kan:.3f}/{sat_kan:.3f}({test_sat_kan:.3f})")

print("Training finished.")

# save the models as .pt files
save_model(kan, model_state_folder, 'logiKAN.pt')

checkpoint directory created: ./model
saving model version 0.0
Epoch 0 | KAN (loss/acc/sat): 0.834/0.274/0.166(0.167)
Epoch 1 | KAN (loss/acc/sat): 0.833/0.285/0.167(0.167)
Epoch 2 | KAN (loss/acc/sat): 0.833/0.295/0.167(0.168)
Epoch 3 | KAN (loss/acc/sat): 0.832/0.308/0.168(0.169)
Epoch 4 | KAN (loss/acc/sat): 0.831/0.317/0.169(0.170)
Epoch 5 | KAN (loss/acc/sat): 0.831/0.326/0.169(0.170)
Epoch 6 | KAN (loss/acc/sat): 0.830/0.337/0.170(0.171)
Epoch 7 | KAN (loss/acc/sat): 0.829/0.346/0.171(0.172)
Epoch 8 | KAN (loss/acc/sat): 0.829/0.357/0.171(0.172)
Epoch 9 | KAN (loss/acc/sat): 0.828/0.365/0.172(0.173)
Epoch 10 | KAN (loss/acc/sat): 0.827/0.371/0.173(0.174)
Epoch 11 | KAN (loss/acc/sat): 0.826/0.386/0.174(0.175)
Epoch 12 | KAN (loss/acc/sat): 0.826/0.396/0.174(0.175)
Epoch 13 | KAN (loss/acc/sat): 0.825/0.408/0.175(0.176)
Epoch 14 | KAN (loss/acc/sat): 0.824/0.419/0.176(0.177)
Epoch 15 | KAN (loss/acc/sat): 0.824/0.429/0.176(0.178)
Epoch 16 | KAN (loss/acc/sat): 0.823/0.438/0.177(0.

In [15]:
def load_model_state(infer_model, model_save_folder, model_name):
    """
    Load the model from disk.
    """
    checkpoint = torch.load(os.path.join(model_save_folder, model_name), 
                            map_location=device)
    infer_model.load_state_dict(checkpoint)
    infer_model.eval()
    return infer_model

# Load the model
logiKNet_infer = KAN(width=[18, 10, 6], grid=5, k=3, seed=42, device=device)
logiKNet_infer = load_model_state(logiKNet_infer, model_state_folder, 'logiKNet.pt')
# Test the model
with torch.no_grad():
    for data, labels in test_loader:
        logits = logiKNet_infer(data)
        preds = torch.argmax(logits, dim=1)
        print(f"Predictions: {preds}, Labels: {labels}")
        
        logiKNet_infer_acc = compute_accuracy(test_loader, logiKNet_infer)
        print(f"Test accuracy of the loaded model: {logiKNet_infer_acc.item()}")
        break

checkpoint directory created: ./model
saving model version 0.0
Predictions: tensor([2, 2, 0,  ..., 2, 4, 2]), Labels: tensor([2, 0, 0,  ..., 2, 4, 2])
Test accuracy of the loaded model: 0.7819228768348694


/tmp/ipykernel_1298114/1172964025.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(os.path.join(model_save_folder, model_name),


**hierarchical-logiKNet**

In [12]:
def compute_sat_levels(loader, P):
	sat_level  = 0
	for data, labels in loader:
		x = ltn.Variable("x", data)
		x_MQTT_DDoS_Connect_Flood = ltn.Variable("x_MQTT_DDoS_Connect_Flood", data[labels == 0])
		x_MQTT_DDoS_Publish_Flood = ltn.Variable("x_MQTT_DDoS_Publish_Flood", data[labels == 1])
		x_MQTT_DoS_Connect_Flood = ltn.Variable("x_MQTT_DoS_Connect_Flood", data[labels == 2])
		x_MQTT_DoS_Publish_Flood = ltn.Variable("x_MQTT_DoS_Publish_Flood", data[labels == 3])
		x_MQTT_Malformed_Data = ltn.Variable("x_MQTT_Malformed_Data", data[labels == 4])
		x_Benign = ltn.Variable("x_Benign", data[labels == 5])
		######################
		x_MQTT = ltn.Variable("x_MQTT", data[labels < 5]) 
		######################
		
		sat_level = SatAgg(
			Forall(x_MQTT_DDoS_Connect_Flood, P(x_MQTT_DDoS_Connect_Flood, l_MQTT_DDoS_Connect_Flood)),
			Forall(x_MQTT_DDoS_Publish_Flood, P(x_MQTT_DDoS_Publish_Flood, l_MQTT_DDoS_Publish_Flood)),
			Forall(x_MQTT_DoS_Connect_Flood, P(x_MQTT_DoS_Connect_Flood, l_MQTT_DoS_Connect_Flood)),
			Forall(x_MQTT_DoS_Publish_Flood, P(x_MQTT_DoS_Publish_Flood, l_MQTT_DoS_Publish_Flood)),
			Forall(x_MQTT_Malformed_Data, P(x_MQTT_Malformed_Data, l_MQTT_Malformed_Data)),
			Forall(x_Benign, P(x_Benign, l_Benign)),
			# hierarchical constraints
			Forall(x_MQTT, Not(P(x_MQTT, l_Benign)))
		)
	return sat_level

kan_h = KAN(width=[18, 10, 6], grid=5, k=3, seed=42, device=device)
P_kan_h = ltn.Predicate(LogitsToPredicate(MultiKANModel(kan_h)))

optimizer_kan = torch.optim.Adam(P_kan_h.parameters(), lr=0.001)

for epoch in range(401):
	optimizer_kan.zero_grad()
	sat_kan = compute_sat_levels(train_loader, P_kan_h)
	loss = 1. - sat_kan
	loss.backward()
	optimizer_kan.step()
	train_loss_kan = loss.item()

	# Test the KAN
	test_acc_kan = compute_accuracy(test_loader, kan_h)
	test_sat_kan = compute_sat_levels(test_loader, P_kan_h)

	print(f"Epoch {epoch} | KAN (loss/acc/sat): {train_loss_kan:.3f}/{test_acc_kan:.3f}/{sat_kan:.3f}({test_sat_kan:.3f})")

print("Training finished.")

# save the models as .pt files
save_model(kan_h, model_state_folder, 'hierarchical_logiKNet.pt')


checkpoint directory created: ./model
saving model version 0.0
Epoch 0 | KAN (loss/acc/sat): 0.775/0.275/0.225(0.226)
Epoch 1 | KAN (loss/acc/sat): 0.774/0.287/0.226(0.227)
Epoch 2 | KAN (loss/acc/sat): 0.774/0.300/0.226(0.227)
Epoch 3 | KAN (loss/acc/sat): 0.773/0.312/0.227(0.228)
Epoch 4 | KAN (loss/acc/sat): 0.772/0.324/0.228(0.228)
Epoch 5 | KAN (loss/acc/sat): 0.772/0.336/0.228(0.229)
Epoch 6 | KAN (loss/acc/sat): 0.771/0.348/0.229(0.230)
Epoch 7 | KAN (loss/acc/sat): 0.770/0.354/0.230(0.230)
Epoch 8 | KAN (loss/acc/sat): 0.770/0.360/0.230(0.231)
Epoch 9 | KAN (loss/acc/sat): 0.769/0.364/0.231(0.232)
Epoch 10 | KAN (loss/acc/sat): 0.768/0.368/0.232(0.232)
Epoch 11 | KAN (loss/acc/sat): 0.768/0.381/0.232(0.233)
Epoch 12 | KAN (loss/acc/sat): 0.767/0.387/0.233(0.234)
Epoch 13 | KAN (loss/acc/sat): 0.766/0.395/0.234(0.235)
Epoch 14 | KAN (loss/acc/sat): 0.766/0.403/0.234(0.235)
Epoch 15 | KAN (loss/acc/sat): 0.765/0.410/0.235(0.236)
Epoch 16 | KAN (loss/acc/sat): 0.764/0.418/0.236(0.

In [13]:
def load_model_state(infer_model, model_save_folder, model_name):
    """
    Load the model from disk.
    """
    checkpoint = torch.load(os.path.join(model_save_folder, model_name), 
                            map_location=device)
    infer_model.load_state_dict(checkpoint)
    infer_model.eval()
    return infer_model

# Load the model
hierarchical_logiKNet_infer = KAN(width=[18, 10, 6], grid=5, k=3, seed=42, device=device)
hierarchical_logiKNet_infer = load_model_state(hierarchical_logiKNet_infer, model_state_folder, 'hierarchical_logiKNet.pt')
# Test the model
with torch.no_grad():
    for data, labels in test_loader:
        logits = hierarchical_logiKNet_infer(data)
        preds = torch.argmax(logits, dim=1)
        print(f"Predictions: {preds}, Labels: {labels}")
        
        hierarchical_logiKNet_infer_acc = compute_accuracy(test_loader, hierarchical_logiKNet_infer)
        print(f"Test accuracy of the loaded model: {hierarchical_logiKNet_infer_acc.item()}")
        break

checkpoint directory created: ./model
saving model version 0.0
Predictions: tensor([2, 2, 0,  ..., 2, 4, 2]), Labels: tensor([2, 0, 0,  ..., 2, 4, 2])
Test accuracy of the loaded model: 0.7824236154556274


/tmp/ipykernel_1302988/4219946763.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(os.path.join(model_save_folder, model_name),
